# 1. PyRIT Scan

`pyrit_scan` allows you to run automated security testing and red teaming attacks against AI systems using [scenarios](../scenarios/0_scenarios.ipynb) for strategies and [configuration](../setup/1_configuration.ipynb).

Note in this doc the ! prefaces all commands in the terminal so we can run in a Jupyter Notebook.

## Quick Start

For help:

In [ ]:
!pyrit_scan --help

Starting PyRIT...
usage: pyrit_scan [-h] [--log-level LOG_LEVEL] [--list-scenarios]
                  [--list-initializers] [--database DATABASE]
                  [--initializers INITIALIZERS [INITIALIZERS ...]]
                  [--initialization-scripts INITIALIZATION_SCRIPTS [INITIALIZATION_SCRIPTS ...]]
                  [--strategies SCENARIO_STRATEGIES [SCENARIO_STRATEGIES ...]]
                  [--max-concurrency MAX_CONCURRENCY]
                  [--max-retries MAX_RETRIES] [--memory-labels MEMORY_LABELS]
                  [scenario_name]

PyRIT Scanner - Run security scenarios against AI systems

Examples:
  # List available scenarios and initializers
  pyrit_scan --list-scenarios
  pyrit_scan --list-initializers

  # Run a scenario with built-in initializers
  pyrit_scan foundry_scenario --initializers openai_objective_target

  # Run with custom initialization scripts
  pyrit_scan encoding_scenario --initialization-scripts ./my_config.py

  # Run specific strategies
  pyri

### Discovery

List all available scenarios:

In [ ]:
!pyrit_scan --list-scenarios

Starting PyRIT...
Loading PyRIT modules...

Available Scenarios:

  content_harms_scenario
    Class: ContentHarmsScenario
    Description:
      Content Harms Scenario implementation for PyRIT. This scenario contains
      various harm-based checks that you can run to get a quick idea about
      model behavior with respect to certain harm categories.
    Aggregate Strategies:
      - all
    Available Strategies (7):
      hate, fairness, violence, sexual, harassment, misinformation, leakage
    Default Strategy: all

  cyber_scenario
    Class: CyberScenario
    Description:
      Cyber scenario implementation for PyRIT. This scenario tests how willing
      models are to exploit cybersecurity harms by generating malware. The
      CyberScenario class contains different variations of the malware
      generation techniques.
    Aggregate Strategies:
      - all
    Available Strategies (2):
      single_turn, multi_turn
    Default Strategy: all

  encoding_scenario
    Class: Encod

**Tip**: You can also discover user-defined scenarios by providing initialization scripts:

```shell
pyrit_scan --list-scenarios --initialization-scripts ./my_custom_initializer.py
```

This will load your custom scenario definitions and include them in the list.

## Initializers

PyRITInitializers are how you can configure the CLI scanner. PyRIT includes several built-in initializers you can use with the `--initializers` flag.

The `--list-initializers` command shows all available initializers. Initializers are referenced by their filename (e.g., `objective_target`, `objective_list`, `simple`) regardless of which subdirectory they're in.

List the available initializers using the --list-initializers flag.

In [ ]:
!pyrit_scan --list-initializers

Starting PyRIT...

Available Initializers:

  objective_list
    Class: ScenarioObjectiveListInitializer
    Name: Simple Objective List Configuration for Scenarios
    Execution Order: 10
    Required Environment Variables: None
    Description:
      Simple Objective List Configuration for Scenarios

  openai_objective_target
    Class: ScenarioObjectiveTargetInitializer
    Name: Simple Objective Target Configuration for Scenarios
    Execution Order: 10
    Required Environment Variables:
      - DEFAULT_OPENAI_FRONTEND_ENDPOINT
      - DEFAULT_OPENAI_FRONTEND_KEY
    Description:
      This configuration sets up a simple objective target for scenarios using
      OpenAIChatTarget with basic settings. It initializes an openAI chat
      target using the OPENAI_CLI_ENDPOINT and OPENAI_CLI_KEY environment
      variables.


Total initializers: 2


### Running Scenarios

You need a single scenario to run, you need two things:

1. A Scenario. Many are defined in `pyrit.scenario.scenarios`. But you can also define your own in initialization_scripts.
2. Initializers (which can be supplied via `--initializers` or `--initialization-scripts`). Scenarios often don't need many arguments, but they can be configured in different ways. And at the very least, most need an `objective_target` (the thing you're running a scan against).
3. Scenario Strategies (optional). These are supplied by the `--scenario-strategies` flag and tell the scenario what to test, but they are always optional. Also note you can obtain these by running `--list-scenarios`

Basic usage will look something like:

```shell
pyrit_scan <scenario> --initializers <initializer1> <initializer2> --scenario-strategies <strategy1> <strategy2>
```

You can also override scenario parameters directly from the CLI:

```shell
pyrit_scan <scenario> --max-concurrency 10 --max-retries 3 --memory-labels '{"experiment": "test1", "version": "v2"}'
```

Or concretely:

```shell
!pyrit_scan foundry_scenario --initializers simple openai_objective_target load_default_datasets --scenario-strategies base64
```

Example with a basic configuration that runs the Foundry scenario against the objective target defined in `openai_objective_target` (which just is an OpenAIChatTarget with `DEFAULT_OPENAI_FRONTEND_ENDPOINT` and `DEFAULT_OPENAI_FRONTEND_KEY`), along with `load_default_datasets` which loads the required datasets into memory.

In [ ]:
!pyrit_scan foundry_scenario --initializers openai_objective_target load_default_datasets --strategies base64

Starting PyRIT...
Loading PyRIT modules...
Running 1 initializer(s)...

Running scenario: foundry_scenario


Error: 'charmap' codec can't encode character '\U0001f4ca' in position 41: character maps to <undefined>



Executing Foundry Scenario:   0%|          | 0/2 [00:00<?, ?attack/s]WARNING:pyrit.prompt_target.openai.openai_target:BadRequestError request_id=None is_content_filter=True payload={"error": {"message": "The response was filtered due to the prompt triggering Azure OpenAI's content management policy. Please modify your prompt and retry. To learn more about our content filtering p
ERROR:pyrit.exceptions.exception_classes:BadRequestException encountered: Status Code: 400, Message: {'error': {'message': "The response was filtered due to the prompt triggering Azure OpenAI's content management policy. Please modify your prompt and retry. To learn more about our content filtering policies please read our documentation: https://go.microsoft.com/fwlink/?linkid=2198766", 'type': None, 'param': 'prompt', 'code': 'content_filter', 'status': 400, 'innererror': {'code': 'ResponsibleAIPolicyViolation', 'content_filter_result': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered

Or with all options and multiple initializers and multiple strategies:

```shell
pyrit_scan foundry_scenario --database InMemory --initializers simple objective_target objective_list load_default_datasets --scenario-strategies easy crescendo
```

You can also override scenario execution parameters:

```shell
# Override concurrency and retry settings
pyrit_scan foundry_scenario --initializers simple objective_target load_default_datasets --max-concurrency 10 --max-retries 3

# Add custom memory labels for tracking (must be valid JSON)
pyrit_scan foundry_scenario --initializers simple objective_target load_default_datasets --memory-labels '{"experiment": "test1", "version": "v2", "researcher": "alice"}'
```

Available CLI parameter overrides:
- `--max-concurrency <int>`: Maximum number of concurrent attack executions
- `--max-retries <int>`: Maximum number of automatic retries if the scenario raises an exception
- `--memory-labels <json>`: Additional labels to apply to all attack runs (must be a JSON string with string keys and values)

You can also use custom initialization scripts by passing file paths. It is relative to your current working directory, but to avoid confusion, full paths are always better:

```shell
pyrit_scan garak.encoding_scenario --initialization-scripts ./my_custom_config.py
```

#### Using Custom Scenarios

You can define your own scenarios in initialization scripts. The CLI will automatically discover any `Scenario` subclasses and make them available:


In [ ]:
# my_custom_scenarios.py
from pyrit.common.apply_defaults import apply_defaults
from pyrit.scenario import Scenario
from pyrit.scenario.core.scenario_strategy import ScenarioStrategy


class MyCustomStrategy(ScenarioStrategy):
    """Strategies for my custom scenario."""

    ALL = ("all", {"all"})
    Strategy1 = ("strategy1", set[str]())
    Strategy2 = ("strategy2", set[str]())


@apply_defaults
class MyCustomScenario(Scenario):
    """My custom scenario that does XYZ."""

    @classmethod
    def get_strategy_class(cls):
        return MyCustomStrategy

    @classmethod
    def get_default_strategy(cls):
        return MyCustomStrategy.ALL

    def __init__(self, *, scenario_result_id=None, **kwargs):
        # Scenario-specific configuration only - no runtime parameters
        super().__init__(
            name="My Custom Scenario",
            version=1,
            strategy_class=MyCustomStrategy,
            scenario_result_id=scenario_result_id,
        )
        # ... your scenario-specific initialization code

    async def _get_atomic_attacks_async(self):
        # Build and return your atomic attacks
        return []

Then discover and run it:

```shell
# List to see it's available
pyrit_scan --list-scenarios --initialization-scripts ./my_custom_scenarios.py

# Run it with parameter overrides
pyrit_scan my_custom_scenario --initialization-scripts ./my_custom_scenarios.py --max-concurrency 10
```

The scenario name is automatically converted from the class name (e.g., `MyCustomScenario` becomes `my_custom_scenario`).